# A minimal version of asprin

The goal of this project is to implement a minimal version of the system *asprin*.

Before you start, we recommend you to read sections 4 and 5 of the paper [1], and 
do the following little project on multi-shot solving:
* [../multi-shot-shell/multi-shot-shell.ipynb](../multi-shot-shell/multi-shot-shell.ipynb)

We introduce *asprin* below, but if you are interested, you can find more information at:
* the paper [2]
* the repository https://github.com/potassco/asprin
* these [slides](https://github.com/potassco/asprin/releases/download/v1/asprin-slides-btw17.pdf)
* the [Potassco Guide](https://github.com/potassco/guide/releases/download/v2.2.0/guide.pdf)


[1] [Kaminski, R., Romero, J., Schaub, T., & Wanko, P. (2023). How to Build Your Own ASP-based System?! Theory Pract. Log. Program., 23(1), 299–361.](https://www.cs.uni-potsdam.de/wv/publications/DBLP_journals/tplp/KaminskiRSW23.pdf)

[2] [Brewka, G., Delgrande, J. P., Romero, J., & Schaub, T. (2023). A general framework for preferences in answer set programming. Artif. Intell., 325, 104023.](https://www.cs.uni-potsdam.de/wv/publications/DBLP_journals/ai/BrewkaDRS23.pdf) 

# Clingo with optimization statements 

In *clingo* we can write logic programs with optimization statements and compute their optimal answer sets.

The following *clingo* program:

In [52]:
%%file example1.lp
dom(1..2).
1 { a(X) : dom(X) }.
#show a/1.

Overwriting example1.lp


has 3 answer sets: `{a(1)}`, `{a(2)}` and `{a(1),a(2)}`:

In [66]:
! clingo example1.lp 0 -V0

a(2)
a(1)
a(1) a(2)
SATISFIABLE


If we add a minimize statement:

In [54]:
%%file minimize.lp
#minimize{ 1,X : a(X) }.

Overwriting minimize.lp


then the resulting program has 2 **optimal** answer sets: `{a(1)}` and `{a(2)}`:

In [55]:
! clingo example1.lp minimize.lp --opt-mode=optN 0 -V0 --quiet=1

a(2)
Optimization: 1
a(1)
Optimization: 1
OPTIMUM FOUND


Here, option `--quiet=1` tells clingo to print only the optimal answer sets.

#### Formalities

A *clingo* program with minimize statements can be seen as a pair of:
* a logic program $P$
* a minimize statement $M$

Let $\mathcal{A}$ be the set of (ground) atoms occurring in the ground instantiation of $P$.

The minimize statement $M$ declares an irreflexive and transitive relation $\succ_M \subseteq \mathcal{A}\times\mathcal{A}$ 
(also called a [strict partial order](https://en.wikipedia.org/wiki/Partially_ordered_set#strict_partial_order)) where
* $X \succ_M Y$ if the weight of $X$ according to $M$ is less than the weight of $Y$ according to $M$

If $X \succ_M Y$ we say that $X$ is *better* than $Y$ wrt $M$.
  
Then, a set of atoms $X$ is an optimal answer set of $P$ and $M$ if:
* $X$ is an answer set of $P$, and
* there is no answer set $Y$ of $P$ such that $Y \succ_M X$

In our example:
* the set $\mathcal{A}$ is `{a(1),a(2)}`,
* the logic program $P$ has three answer sets `{a(1)}`, `{a(2)}` and `{a(1),a(2)}`,
* the minimize statement declares the strict partial order where:
  - `{}`$\succ_M$ `{a(1)}`, `{}`$\succ_M$ `{a(2)}`, `{}`$\succ_M$ `{a(1),a(2)}`,
  - `{a(1)}`$\succ_M$ `{a(1),a(2)}`,
  - `{a(2)}`$\succ_M$ `{a(1),a(2)}`, and
* the answer set `{a(1),a(2)}` is not optimal because both `{a(1)}` and `{a(2)}` are answer sets better than `{a(1),a(2)}` wrt $M$,
* while `{a(1)}` and `{a(2)}` are optimal answer sets because there is no answer set better than them (since `{}` is not an answer set).

# The ASP system *asprin*

The system *asprin* extends *clingo* by preference specifications, 
allowing more general optimization statements: subset preferences, conditional prefereces, their combination, and much more...

Comparing both systems:
* in *clingo* we can write logic programs with *optimization statements* and compute their optimal answer sets, while
* in *asprin* we can write logic programs with *preference specifications* and compute their optimal answer sets.

A nice feature of *asprin* is that we can define new types of preferences simply writing a logic program.

#### Formalities

An *asprin* program can be seen as a pair of:
* a logic program $P$
* a preference specification $S$

where now $S$ declares the irreflexive and transitive relation $\succ_S \subseteq \mathcal{A}\times\mathcal{A}$.

Optimal answer sets are defined just as above. 

A set of atoms $X$ is an optimal answer set of $P$ and $S$ if:
* $X$ is an answer set of $P$, and
* there is no answer set $Y$ of $P$ such that $Y \succ_S X$

### The same example in *asprin*

In asprin, instead of the previous minimize statement, we can write this *preference specification*:

In [28]:
%%file preference1.lp
#preference(p1,less(weight)) { 1,X :: a(X) : dom(X) }.
#optimize(p1).

Overwriting preference1.lp


The **first line** says that the preference statement `p1` is of type `less(weight)` and has the preference element `1,X :: a(X) : dom(X)`.

This statement declares the strict partial order $\succ_{\texttt{p1}}$, 
that is the same as the order declared by the minimize statement $M$ from above, i.e., $\succ_{\texttt{p1}} = \succ_M$.

The **second line** tells *asprin* to compute optimal models wrt `p1`.

To use *asprin* in this notebook, we clone it from its repository:

In [ ]:
! git clone https://github.com/potassco/asprin.git

Normally, we would install it with:
* `conda install -c potassco asprin --yes`

but today (23.5.2024) the latest version available via conda is not compatible with the latest version of *clingo* used in this notebook.

In [63]:
! python asprin/asprin/asprin.py example1.lp preference1.lp 0

asprin version 3.1.2beta
Reading from example1.lp ...
Solving...
Answer: 1
a(2)
OPTIMUM FOUND
Answer: 2
a(1)
OPTIMUM FOUND

Models       : 2
  Optimum    : yes
  Optimal    : 2
Calls        : 7
Time         : 0.080s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.076s


Observe how we obtain the same optimal answer sets as above with the minimize statement.

## Extending the preference specification

Consider now this preference specification:

In [38]:
%%file preference2.lp
#preference(p1,less(weight)) { 1,X :: a(X) : dom(X) }.
#preference(p2,      subset) {        a(X) : dom(X) }.

#const s=p2.
#optimize(s).

Overwriting preference2.lp


The first line is the same as above.

The **second line** says that the preference statement `p2` is of type `subset` and has the preference element `a(X) : dom(X)`.

This declares the strict partial order $\succ_\texttt{p2}$ where $X \succ_\texttt{p2} Y$ if 
`{a(1),a(2)}` $\cap \ X \subset$ `{a(1),a(2)}` $\cap \ Y$.

It turns out that $\succ_\texttt{p2}$ is the same as $\succ_\texttt{p1}$ and as $\succ_M$. You can check this by hand.

Hence, the optimal models wrt `p2` are the same as before. 

The **last lines** says tell *asprin* to compute those optimal models.

In [40]:
! python asprin/asprin/asprin.py example1.lp preference2.lp 0

asprin version 3.1.2beta
Reading from example1.lp ...
Solving...
Answer: 1
a(2)
OPTIMUM FOUND
Answer: 2
a(1)
OPTIMUM FOUND

Models       : 2
  Optimum    : yes
  Optimal    : 2
Calls        : 7
Time         : 0.075s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.074s


We obtain the same answer sets if we set the constant `s` to be `p1` (adding `-c s=p1` to the previous call).

We can now extend the base program with these rules:

In [41]:
%%file example2.lp
dom(3).
a(3) :- a(2).
a(2) :- a(3).

Overwriting example2.lp


and we obtain 3 answer sets: 

In [42]:
! clingo example1.lp example2.lp 0 -V0

a(1)
a(3) a(2)
a(3) a(2) a(1)
SATISFIABLE


Only the first one is minimal wrt the minimize statement or wrt `p1`:

In [43]:
! clingo example1.lp example2.lp minimize.lp --opt-mode=optN 0 -V0 --quiet=1

a(1)
Optimization: 1
OPTIMUM FOUND


In [44]:
! python asprin/asprin/asprin.py example1.lp example2.lp preference2.lp 0 -c s=p1

asprin version 3.1.2beta
Reading from example1.lp ...
Solving...
Answer: 1
a(1)
OPTIMUM FOUND

Models       : 1
  Optimum    : yes
  Optimal    : 1
Calls        : 4
Time         : 0.070s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.069s


but both `{a(1)}` and `{a(2),a(3)}` are optimal wrt `p2`:

In [45]:
! python asprin/asprin/asprin.py example1.lp example2.lp preference2.lp 0

asprin version 3.1.2beta
Reading from example1.lp ...
Solving...
Answer: 1
a(1)
OPTIMUM FOUND
Answer: 2
a(3) a(2)
OPTIMUM FOUND

Models       : 2
  Optimum    : yes
  Optimal    : 2
Calls        : 7
Time         : 0.073s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.073s


How can this be the case?

The reason is that the order declared by `p2` is different from the order declared by either `p1` or $M$.

Now we have that $X \succ_\texttt{p2} Y$ if 
`{a(1),a(2),a(3)}` $\cap \ X \subset$ `{a(1),a(2),a(3)}` $\cap \ Y$.

But then, `{a(1)}` $\not\succ_\texttt{p2}$ `{a(2),a(3)}`. 
This makes sense, since `{a(1)}` is not a subset of `{a(2),a(3)}`.

On the other hand, `{a(1)}` $\succ_\texttt{p1}$ `{a(2),a(3)}`.
This makes sense, since the weight of `{a(1)}` is `1`, and this is less than the weight of `{a(2),a(3)}`, that is `2`.


Then, `{a(2),a(3)}` is optimal wrt `p2` because none of its subsets (`{}`, `{a(2)}` or `{a(3)}`) is an answer set, 
but it is not optimal wrt `p1` because `{a(1)}` is an answer set better than it wrt `p1`.

This kind of subset preferences are very useful in practice and show up often in real-world applications.

# Task


The task of this project is to implement a minimal version of *asprin*, called *mini-asprin*.

The input to *mini-asprin* should be:
- some *clingo* program including minimize statements, and
- some *preference* program re-interpreting those minimize statements (see below).

Given that input, 
*mini-asprin* should print the answer sets of the *clingo* program 
that are optimal wrt the re-interpretation of the minimize statements. 

For example, if:
* the input consists of the files `example1.lp example2.lp minimize.lp`, and
* the preference program (in `less-weight.lp`) interprets minimize statements as usual,

then we should obtain the unique optimal answer set `{a(1)}`:

In [24]:
! clingo example1.lp example2.lp minimize.lp --output=reify | python mini-asprin.py - less-weight.lp 0

clingo version 5.7.1
Reading from - ...
Solving...
Answer: 1
a(1)
Solving...
OPTIMUM FOUND
Solving...
UNSATISFIABLE

Models       : 1
Calls        : 3
Time         : 0.002s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.002s


But if the preference program (in `subset.lp`) interprets the minimize statement as a subset preference, 
then we should obtain the two optimal answer sets `{a(1)}` and `{a(2), a(3)}`:

In [25]:
! clingo example1.lp example2.lp minimize.lp --output=reify | python mini-asprin.py - subset.lp 0

clingo version 5.7.1
Reading from - ...
Solving...
Answer: 1
a(1)
Solving...
OPTIMUM FOUND
Solving...
Answer: 1
a(2) a(3)
Solving...
OPTIMUM FOUND
Solving...
UNSATISFIABLE

Models       : 2
Calls        : 5
Time         : 0.004s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.004s


We recommend you to extend the initial script that comes with this notebook. As you can see, it uses meta-programming.

In its current form, the script loads the input and the meta-encoding, grounds both together and solves.

It computes normal answer sets of the input program.

In [26]:
! cat mini-asprin.py

import sys
from clingo.application import clingo_main, Application
from clingo.script import enable_python
from clingo.symbol import Number, Function

enable_python()

META = """
conjunction(B) :- literal_tuple(B),
        hold(L,0) : literal_tuple(B, L), L > 0;
    not hold(L,0) : literal_tuple(B,-L), L > 0.

body(normal(B)) :- rule(_,normal(B)), conjunction(B).
body(sum(B,G))  :- rule(_,sum(B,G)),
    #sum { W,L :     hold(L,0), weighted_literal_tuple(B, L,W), L > 0 ;
           W,L : not hold(L,0), weighted_literal_tuple(B,-L,W), L > 0 } >= G.

  hold(A,0) : atom_tuple(H,A)   :- rule(disjunction(H),B), body(B).
{ hold(A,0) : atom_tuple(H,A) } :- rule(     choice(H),B), body(B).

#show.
#show T : output(T,B), conjunction(B).
"""

HOLD = """
hold(A,m) :- A = @get_hold().
"""

DELETE = """
:- hold(A,0) : hold(A,m);
   hold(A,m) : hold(A,0).
"""

class MiniAsprinApp(Application):

    def main(self, ctl, files):
        for path in files:
            ctl.load(path)
        if not files:

As you can see, 
the script uses a modification of the common meta-encoding,
where true atoms are represented by `hold(A,0)` instead of `hold(A)`.

The final `mini-asprin` will often compute many answer sets.
We can:
* assign a number greater than 0 to each of them: 1, 2, ..., and
* represent the true atoms in answer set `M` by `hold(A,M)`.

We can use the program `HOLD` for that.

Program `DELETE` can be used to eliminate the answer set `m` 
from our program, where `m` should be the number of an answer set.